# Part 1: Simple RSA (Encryption and Decryption)

RSA is a asymmetric public-key encryption algorithm.
In RSA, **d** is private key. **e** and **N** are public keys. Any two parties willing to communicate with each other will need to use these keys. Messages will be encrypted using the public keys of the partner. However, the message can only be decrypted using the private key of the intended receiver and no one else.


---

*RSA Key Generation:*

1.   Two large prime numbers **p** and **q** are randomly generated.
2.   N is calculated using **N=pq**.
3.   phi_N calculated using **phi_N=(p-1)(q-1)**.
4.   Public key **e** is generated such that it is **not a factor of phi_N**.
5.   Private key d calculated using **d=e^-1 mod phi_N**

*RSA Encryption:*

The messages are encrypted for the partner using the following formula: **Encrypted_Message = PlainText^e mod N** , where e and N are the public key of the partner to whom the cipher text will be send to.


*RSA Decryption:*

The messages are decrypted using the following formula:
  **Decrypted Message = Encrypted_Messaget^d mod N**, where e and N are the public key of the partner who received the cipher text (Encrypted_Message). To decrypt the ciphertext that obtained from my partner, I will use my private key d and my public key N.



---


RSA Digital Signature Scheme:
Signature Scheme is needed to ensure that they are receiving data from the corrrect person. For this project, the overall working process of the scheme is as follows:

1.   Both partners will create a signature on their messages using **S=M^d mod N**.
2.   They will share the message(M) and the corresponding signature(S).
3.   Partners will then verify the validity of the signature by computing **M1=S^e mod N** on their partner's signature. If the M1 matches the message(M) send by their partner then it can be said that the signature is valid.









## Generating Public and Private Keys

In [1]:
import random, math
random.seed(0)  #using random seed to ensure this whole project will give the same result everytime

#GCD function
def gcd(a, b):
    while b != 0:
        (a, b) = (b, a % b)
    return a

#Checking if the number is prime function
def is_Prime(n):
    if (n % 2 == 0 and n > 2) or (n < 2):
        return False
    for i in range(3, int(math.sqrt(n)), 2):
        if n % i == 0:
            return False
    return True

#Creating the range for 16 digit number generation
highestbit = int('1000000000000000', 2) #generating lowest and highest integers of 16 bit digits
lowestbit =  int('1111111111111111', 2)

#Generating two different large prime P and Q
primes = [i for i in range(highestbit,lowestbit) if is_Prime(i)]
p = random.choice(primes)
q = random.choice(primes)
if(p==q): q=random.choice(primes)

#Calculating N and Phi_N
N=p*q
phi_N = (p-1)*(q-1)


print('p =', p, 'P is Prime?:',is_Prime(p))
print('q =', q,'Q is Prime?:',is_Prime(q))
print('N =', N)
print('ϕ(N) =',phi_N)

#Generating the public key e
e=random.randrange(1, phi_N) #this will generate values between 1 and phi_N(value greater than 1 and less than phi_N)
while(gcd(phi_N, e)!=1): e=random.randrange(1, phi_N)
print('e =', e, '  GCD(phi_N, e) =', gcd(phi_N, e))

#Generating the private key d
def extendedEuclideanGCD(a, b):
    x = 1; y = 0; d = a; r = 0; s = 1; t = b
    while(t > 0):
        q = d//t
        u = x-q*r
        v = y-q*s
        w = d-q*t
        x = r
        y = s
        d = t
        r = u
        s = v
        t = w
    return d, x, y

result = extendedEuclideanGCD(e, phi_N)    #d=e^-1 mod phi_N
d=result[1]

if(d<0): d=(phi_N+result[1]) %phi_N

print('The public keys are N =',N,'and e =',e,'The private key is d =',d)

p = 49463 P is Prime?: True
q = 51059 Q is Prime?: True
N = 2525531317
ϕ(N) = 2525430796
e = 173879093   GCD(phi_N, e) = 1
The public keys are N = 2525531317 and e = 173879093 The private key is d = 1539027341


##Encryption/Decryption:

In [3]:
#Square and Multiply Algorithm
def square_multiply(a, b, c):
  multiple=1
  bits=bin(b).replace('0b', '')
  length=len(bits)
  result = a % c
  multiple=1
  i=0
  for bit in reversed(bits):  #Reversed funtn is used as I want to interate the bits from backwards
    if(i!=0):
      result = result*result %c  #DON'T WANT THIS FOR THE FIRST BIT AS THE A%C IS ALREADY CALCULATED OUTSIDE THE LOOP
    i=i+1                        #THIS i IS NEEDED TO PREVENT THE ABOVE STATEMENT FROM BEING RUN ONLY DURING THE FIRST BIT
    if int(bit) != 0:          #SOMETHING WAS WRoNG WHEN I USED bit!=0 instead of int(bit). bit !=0 would be true for bit=0 thats why using int()
      multiple=multiple*result    #Whenever bit=1 i multiply the values
  return multiple%c

### A) Encryption:
Sending an encrypted message to my Project partner

In [2]:
def encryption(N, power, plain_text):

  # Turning my message into 3 character chunks
  chunk_text = [plain_text[i:i+3] for i in range(0, len(plain_text), 3)]
  print("My Message: ",my_string)
  print("Messages into chunks: ",chunk_text)

  # Turning each individual text chunks to their corresponding hexadecimal chunks
  chunk_hex = []
  for chunks in chunk_text:
    #print(chunks)
    text_to_int = chunks.encode('utf-8')
    hex_str = text_to_int.hex()
    #print(hex_str)
    chunk_hex.append(hex_str)
  print("Turning Chunks into HEX Values:",chunk_hex)

  # Turning each hexadecimal chunks to integer chunks
  chunk_integers = []
  for chunks in chunk_hex:
    #print(chunks)
    hex_to_int = int(chunks, 16)
    chunk_integers.append(hex_to_int)
  print("Turning Hex values into integers:",chunk_integers)

  # Turning each integer chunks to ciphertext integers using the formula c= m^e mod N. M is the integer chunks. e and N are partner's public keys
  encrypted_message = []   #to encrypt the msg, the N and E value that will be used to encrypt will have to be of your partner
  for chunks in chunk_integers:
    #print(chunks)
    cipher_val = square_multiply(chunks, power, N)  # c = m^e mod N,  e = power
    encrypted_message.append(cipher_val)
  print("Encrypted Message: ",encrypted_message)


  return encrypted_message

In [4]:
my_string = "I wish to see the Aurora Borealis someday."
partner_N =	1724525471
partner_e =	1548545119
my_message_encrypted = encryption(partner_N, partner_e, my_string)

My Message:  I wish to see the Aurora Borealis someday.
Messages into chunks:  ['I w', 'ish', ' to', ' se', 'e t', 'he ', 'Aur', 'ora', ' Bo', 'rea', 'lis', ' so', 'med', 'ay.']
Turning Chunks into HEX Values: ['492077', '697368', '20746f', '207365', '652074', '686520', '417572', '6f7261', '20426f', '726561', '6c6973', '20736f', '6d6564', '61792e']
Turning Hex values into integers: [4792439, 6910824, 2126959, 2126693, 6627444, 6841632, 4289906, 7303777, 2114159, 7497057, 7104883, 2126703, 7169380, 6388014]
Encrypted Message:  [710642559, 1505287012, 526104350, 1349473478, 1167983707, 441756265, 1500779454, 673042567, 545545119, 351421923, 1620087107, 1020165487, 188114433, 927390040]


### B) Decryption:
Decrypting the message received from my Project partner

In [6]:
def decryption(N, power, cipher_text):

  ciphertext = []
  for chunks in cipher_text:  #the d and N value are the values of mine.
    #print(chunks)
    cipher_val = square_multiply(chunks, power, N)  #  m = c^d mod N, here d=power
    ciphertext.append(cipher_val)
  print("The cipher values from partener converted to integer:", ciphertext)

  # Turning each integers to hexadecimal chunks
  partner_hexadecimal_chunks = []
  for chunks in ciphertext:
    int_to_hex = hex(chunks)
    partner_hexadecimal_chunks.append(int_to_hex)
  print("The hexadecimal values from those integer:", partner_hexadecimal_chunks)

  # Converting the hex values to their corresponding strings.
  partner_string_text = []
  for chunks in partner_hexadecimal_chunks:
    #print(chunks)
    hex_string = chunks.replace('0x', '')  #removing the 'Ox'
    byte_string = bytes.fromhex(hex_string).decode('utf-8')  #converting the hex values to string
    partner_string_text.append(byte_string)  #attaching each items to the list
  print("The string chunks from hexadecimal:", partner_string_text)

  #join the text to create a complete text
  decrypted_message = "".join(partner_string_text)
  print("The Decrypted message:", decrypted_message)

  return decrypted_message

In [7]:
cipher_text_received = 	 [564941312, 2438733904, 459565689, 856100212, 1773844201, 1228048082, 27760366, 1587440285, 30142684]
my_N = N #Here this N is my public Key

partner_message_decrypted = decryption(my_N, d, cipher_text_received)

The cipher values from partener converted to integer: [4550243, 7502192, 7629167, 7217263, 6692982, 6910565, 7217270, 6910561, 28265]
The hexadecimal values from those integer: ['0x456e63', '0x727970', '0x74696f', '0x6e206f', '0x662076', '0x697265', '0x6e2076', '0x697261', '0x6e69']
The string chunks from hexadecimal: ['Enc', 'ryp', 'tio', 'n o', 'f v', 'ire', 'n v', 'ira', 'ni']
The Decrypted message: Encryption of viren virani


#Part 2: Signature/Verification

##A) Signature:

My Digital Signature is created using **S=M^d mod N** with my d and N value. M is the message.  


In [8]:
# Signature is created by Me
my_sign = "Fahima Noor"
my_signature = encryption(my_N, d, my_sign)

My Message:  I wish to see the Aurora Borealis someday.
Messages into chunks:  ['Fah', 'ima', ' No', 'or']
Turning Chunks into HEX Values: ['466168', '696d61', '204e6f', '6f72']
Turning Hex values into integers: [4612456, 6909281, 2117231, 28530]
Encrypted Message:  [950301236, 1743847582, 2081717385, 1043079059]


## B) Verification:

My Partner's Signature is verified using **M1=S^e mod N** with their e and N value. I will check whether their message M on which they have signed on matches the M1 value that I have obtained using their Signature.

In [9]:
# verfiy the signature
partner_signed_message = "viren virani"
partner_Signature = [944279938, 511933836, 944279938, 1151309101]
partner_N =	1724525471
partner_e = 1548545119

partner_signature_decrypted = decryption(partner_N, partner_e, partner_Signature)  #M1=S^e mod N


if(partner_signed_message == partner_signature_decrypted): result=True
else: result= False

print("Is The Signature Valid:", result)

The cipher values from partener converted to integer: [7760242, 6647328, 7760242, 6385257]
The hexadecimal values from those integer: ['0x766972', '0x656e20', '0x766972', '0x616e69']
The string chunks from hexadecimal: ['vir', 'en ', 'vir', 'ani']
The Decrypted message: viren virani
Is The Signature Valid: True
